# Importing modules

In [1]:
#import h5py
from keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Add, Dense, Activation,\
    ZeroPadding2D, BatchNormalization, Flatten,\
    Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model, Sequential
import keras.backend as K

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

%matplotlib inline
np.random.seed(1)


/home/b40153/github/emg_mc_venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Defining useful functions

In [2]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def max_min_normalization(data_array):
    rows = data_array.shape[0]
    cols = data_array.shape[1]
    
    temp_array = np.zeros((rows,cols))
    col_min = data_array.min(axis=0)
    col_max = data_array.max(axis=0)

    for i in range(0,rows,1):
        for j in range(0,cols,1):
            temp_array[i][j] = (data_array[i][j]-col_min[j])/(col_max[j]-col_min[j])
    return temp_array


def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

# Creating model

In [3]:
def CNN_semg(input_shape=(16,10,1), classes=53):   
    model = Sequential()
    "block 1"
    "32 filters,  a row of the length of number of electrodes,  ReLU"
    model.add(Conv2D(filters=32, kernel_size=(1,10), strides=(1,1),
                     padding='same', name='conv1',
                     input_shape=input_shape))
    "block 2"
    "32 filters 3*3,  ReLU,  average pool 3*3"
    model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1),
                     padding='same', name='conv2'))
    model.add(Activation('relu', name='relu2'))
    model.add(AveragePooling2D((3,3), strides=(2,2), name='pool1'))
    "block 3"
    "64 filters 5*5,  ReLu,  average pool 3*3"
    model.add(Conv2D(filters=64, kernel_size=(5,5), strides=(1,1),
                     padding='same', name='conv3'))
    model.add(Activation('relu', name='relu3'))
    model.add(AveragePooling2D((3,3), strides=(1,1), name='pool2'))
    "block 4"
    "64 filters 5*1,  ReLU"
    model.add(Conv2D(filters=64, kernel_size=(5,1), strides=(1,1),
                     padding='same', name='conv4'))
    model.add(Activation('relu', name='relu4'))
    "block 5"
    "filters 1*1,  softmax loss"
    model.add(Conv2D(filters=32, kernel_size=(1,1), strides=(1,1),
                     padding='same', name='conv5'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu',    name='fc1'))
    model.add(Dense(classes, activation='softmax', name='fc2'))   
    return model

# Loading and preparing data

In [4]:
import nina_helper as nh

subject = 1
db1_path = "/home/b40153/github/emg_mc_venv/emg_mc/datasets/db1"
# Get EMG, repetition and movement data, don't cap maximum length of rest
data_dict = nh.import_db1(db1_path, subject)
info_dict = nh.db1_info()
# Decide window length (200ms window, 100ms increment)
window_len = 20
window_inc = 10
reps = info_dict['rep_labels']
nb_test_reps = 3
# Create a balanced test - training split based on repetition number
train_reps, test_reps = nh.gen_split_balanced(reps, nb_test_reps)
# Normalise EMG data based on training set
emg_data = nh.normalise_emg(data_dict['emg'], data_dict['rep'], train_reps[0, :])
# Window data: x_all data is 4D tensor [observation, time_step, channel, 1] for use with Keras
# y_all: movement label, length: number of windows
# r_all: repetition label, length: number of windows
x_all, y_all, r_all = nh.get_windows(reps, window_len, window_inc,
                                  emg_data, data_dict['move'],
                                  data_dict['rep'])

train_idx = nh.get_idxs(r_all, train_reps[0, :])
test_idx = nh.get_idxs(r_all, test_reps[0, :])
Y_train = nh.to_categorical(y_all[train_idx])
X_train = x_all[train_idx, :, :, :]
Y_test = nh.to_categorical(y_all[test_idx])
X_test = x_all[test_idx, :, :, :]
print ('X_train shape: ', X_train.shape)
print ('Y_train shape: ', Y_train.shape)
print ('X_test shape: ', X_test.shape)
print ('Y_test shape: ', Y_test.shape)

X_train shape:  (33927, 20, 10, 1)
Y_train shape:  (33927, 53)
X_test shape:  (13183, 20, 10, 1)
Y_test shape:  (13183, 53)


# Compile and plot the model

In [5]:
model = CNN_semg(input_shape=(20,10,1), classes=53)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# checkpoint
#filepath="weights-improvement-{epoch:02d}-{acc:.2f}.hdf5"
filepath="best-weights.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, monitor='val_acc',
                             save_best_only=False, mode='max')
callbacks_list = [checkpoint]
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 20, 10, 32)        352       
_________________________________________________________________
conv2 (Conv2D)               (None, 20, 10, 32)        9248      
_________________________________________________________________
relu2 (Activation)           (None, 20, 10, 32)        0         
_________________________________________________________________
pool1 (AveragePooling2D)     (None, 9, 4, 32)          0         
_________________________________________________________________
conv3 (Conv2D)               (None, 9, 4, 64)          51264     
_________________________________________________________________
relu3 (Activation)           (None, 9, 4, 64)          0         
_________________________________________________________________
pool2 (AveragePooling2D)     (None, 7, 2, 64)          0         
__________

# Fit, train & evaluate the model

In [6]:
epochs=20
model.fit(X_train, Y_train, epochs=epochs, batch_size=64, validation_split=0.33,
          callbacks=callbacks_list, verbose=1)
preds_train = model.evaluate(X_train, Y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))
# TODO: evaluate the test set using pre-trained weights (in this case, 
# the model already uses the best weights obtained from the training phase?)
preds_test  = model.evaluate(X_test, Y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))


Train on 22731 samples, validate on 11196 samples
Epoch 1/20
22731/22731 [==============================] - 13s 565us/step - loss: 1.5775 - acc: 0.6418 - val_loss: 1.2790 - val_acc: 0.7044